# 模型融合预测与交易决策引擎

---

### **目标**
本 Notebook 是整个项目的**应用终端**。它的核心职责是模拟实盘环境，对指定的股票执行完整的“数据-预测-决策”流程。

### **工作流程**
1.  **环境设置与目标选定**: 导入库，加载配置，并指定今天要进行预测的目标股票。
2.  **加载已训练构件**: 
    - 自动查找并加载目标股票**最新版本**的 LGBM 和 LSTM 模型。
    - 加载对应的 `StandardScaler`。
    - 加载两个模型完整的 IC 历史记录，用于动态权重计算。
3.  **获取最新特征**: 调用数据处理流水线，获取截至**今天**的最新一行特征数据。
4.  **独立模型预测**: 分别使用 LGBM 和 LSTM 模型对最新特征进行预测。
5.  **动态权重融合**: 
    - 基于 IC 历史计算出 LGBM 和 LSTM 当前的动态权重。
    - 对两个模型的预测值（经过 Z-score 标准化）进行加权融合，得到最终的预测信号。
6.  **风险审批与决策输出**: 
    - 将融合后的信号提交给 `RiskManager` 进行最终审批（如重复信号检查）。
    - 根据审批结果，输出明确的交易决策（如：**【批准开仓：买入】** 或 **【信号被拒：重复信号】**）。

## 1. 环境设置与目标选定

In [2]:
# 文件路径: Prophet.ipynb

import sys
from pathlib import Path

# --- 1. 动态导入 main_train 核心引擎 ---
project_root = str(Path().resolve())
if project_root not in sys.path:
    sys.path.append(project_root)

try:
    from main_train import run_load_config_and_modules, run_single_stock_prediction
    print("--- INFO: 预测引擎 (Prophet) 已初始化。 ---")
except ImportError as e:
    raise ImportError(f"FATAL: 无法从 main_train.py 导入核心函数: {e}")

# --- 2. 加载全局配置和模块 ---
# 在 Notebook 的生命周期中，我们只加载一次
config, modules = run_load_config_and_modules()

if not (config and modules):
    raise RuntimeError("环境初始化失败，请检查配置文件路径和模块导入。")

# --- 3. (可选) 在此手动指定要预测的股票 ---
# 如果留空，则会使用 config.yaml 中 application_settings.prophet_target_ticker 定义的值
TARGET_TICKER_OVERRIDE = '000100.SZ' # 例如: '000100.SZ'

# --- 4. 执行单点预测工作流 ---
if config and modules:
    try:
        run_single_stock_prediction(
            config=config, 
            modules=modules, 
            target_ticker=TARGET_TICKER_OVERRIDE
        )
    except Exception as e:
        print(f"\n--- ERROR: 在执行单点预测时发生严重错误 ---")
        print(e)
        import traceback
        traceback.print_exc()

--- INFO: 预测引擎 (Prophet) 已初始化。 ---
--- 正在初始化环境：加载配置与模块... ---
INFO: 底层并行计算库线程数已设置为: 4
INFO: 项目模块导入成功。
SUCCESS: 配置已从 'configs/config.yaml' 加载。
=== 主工作流：启动单点预测 ===

--- 步骤1：加载所有已训练构件 (自动查找最新版本) ---
  - 正在加载 LGBM 的构件...
    - SUCCESS: LGBM 版本 '20251026' 已加载。
  - 正在加载 LSTM 的构件...
    - SUCCESS: LSTM 版本 '20251026' 已加载。
  - 正在加载 TABTRANSFORMER 的构件...
    - SUCCESS: TabTransformer 版本 '20251026' 已加载。
  - INFO: 正在加载训练时使用的特征列表...
    - SUCCESS: 已从 LGBM 的元数据加载特征列表 (73个)。
SUCCESS: 融合构件已加载。
INFO: 确保运行时数据目录 'data\runtime_data' 已存在。
SUCCESS: RiskManager initialized and connected to 'data\runtime_data\order_history.db'.
--- 步骤1成功完成：所有构件已加载。 ---

--- 步骤2：准备预测所需的数据 ---
INFO: 正在尝试登录 Baostock...
login success!
INFO: Baostock API 登录成功。
INFO: 未在配置中提供有效的 Tushare Token，将跳过 Tushare 相关数据。
  - INFO: 检查全局数据缓存是否为最新...
    - INFO: 缓存不存在或已过期。将重新生成全局数据...
--- 开始生成市场广度数据 ---
  - 正在获取指数成分股列表...


下载成分股日线数据:   0%|          | 0/300 [00:00<?, ?it/s]

  - 正在从 Baostock 下载 sh.600000 (sh.600000) 的日线行情...
  - INFO: 已将 sh.600000 (sh.600000) 的数据缓存至 data\data_cache\raw_ohlcv\raw_sh.600000_2023-10-27_2025-10-26.pkl
  - 正在从 Baostock 下载 sh.600009 (sh.600009) 的日线行情...
  - INFO: 已将 sh.600009 (sh.600009) 的数据缓存至 data\data_cache\raw_ohlcv\raw_sh.600009_2023-10-27_2025-10-26.pkl
  - 正在从 Baostock 下载 sh.600010 (sh.600010) 的日线行情...
  - INFO: 已将 sh.600010 (sh.600010) 的数据缓存至 data\data_cache\raw_ohlcv\raw_sh.600010_2023-10-27_2025-10-26.pkl
  - 正在从 Baostock 下载 sh.600011 (sh.600011) 的日线行情...
  - INFO: 已将 sh.600011 (sh.600011) 的数据缓存至 data\data_cache\raw_ohlcv\raw_sh.600011_2023-10-27_2025-10-26.pkl
  - 正在从 Baostock 下载 sh.600015 (sh.600015) 的日线行情...
  - INFO: 已将 sh.600015 (sh.600015) 的数据缓存至 data\data_cache\raw_ohlcv\raw_sh.600015_2023-10-27_2025-10-26.pkl
  - 正在从 Baostock 下载 sh.600016 (sh.600016) 的日线行情...
  - INFO: 已将 sh.600016 (sh.600016) 的数据缓存至 data\data_cache\raw_ohlcv\raw_sh.600016_2023-10-27_2025-10-26.pkl
  - 正在从 Baostock 下载 sh.600018 (sh.600018) 的日线行

d:\Project\Command\Python\Neural\Wolf_of_Wall_Street\data_process\get_data.py:135: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns_df = closes_df.pct_change().fillna(0)


--- 市场广度数据已生成并缓存至 data\data_cache\market_breadth_2023-10-27_2025-10-26.pkl ---
--- 正在获取市场情绪数据 (恐慌指数) ---
  - 错误 [akshare]: 获取市场情绪数据失败: module 'akshare' has no attribute 'idx_ivix'
    - SUCCESS: 新的全局数据缓存已生成并保存。

--- 正在为 TCL科技 (000100.SZ) 生成【预测用】最新特征 ---
  - 数据窗口: 2023-10-27 to 2025-10-26
  - 正在从本地缓存加载 TCL科技 (sz.000100) 的原始日线数据...
  - INFO: 正在为 'TCL科技的行业指数' (159928.SZ) 获取指数数据...
  - 正在从 Baostock 下载 TCL科技的行业指数 (159928.SZ) 的日线行情...
  - WARNING [BS]: 未能获取到 TCL科技的行业指数 (159928.SZ) 在指定日期范围的数据。
    - 警告: 无法直接获取指数 'TCL科技的行业指数' (159928.SZ)。将尝试在本地合成等权重指数...
    - 错误: 在 API 映射表中未找到指数 'TCL科技的行业指数' (159928.SZ) 的成分股查询接口。
  - INFO: 正在为 '基准指数' (sh.000300) 获取指数数据...
  - 正在从 Baostock 下载 基准指数 (sh.000300) 的日线行情...
  - INFO: 已将 基准指数 (sh.000300) 的数据缓存至 data\data_cache\raw_ohlcv\raw_sh.000300_2023-10-27_2025-10-26.pkl
    - SUCCESS: 已直接从 Baostock 获取到 '基准指数' (sh.000300) 的数据。
INFO: 开始特征计算流水线...
  - [Calculating Features] Running: Technical Indicators...
    - Calculated: ema with params {'length': 10}
    - Cal

Traceback (most recent call last):
  File "d:\Project\Command\Python\Neural\Wolf_of_Wall_Street\main_train.py", line 1279, in run_single_stock_prediction
    stock_info = next((s for s in config.get('stocks_to_process', []) if s['ticker'] == target_ticker), {})
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Project\Command\Python\Neural\Wolf_of_Wall_Street\main_train.py", line 313, in _prophet_generate_decision
    raise RuntimeError(f"预测时生成的特征数据缺少了训练时的关键特征: {missing_features}。")
RuntimeError: 预测时生成的特征数据缺少了训练时的关键特征: {'ema_50', 'signal_ema_50_200'}。
